In [7]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
import mglearn
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV


# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
#Random global
seed = 12345
rng = np.random.default_rng(seed)  # can be called without a seed
rng.random()
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
df = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\RFECved.csv")
df2 = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\scaled.csv")

In [8]:
#Loading the dataset
X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")


# GaussianNB as a baseline

In [15]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

#because only var_smoothing can be 'tuned'
#do a cross validation on different var_smoothing values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)


accuracy=[]
precision=[]
recall=[]
f1_score=[]
confusion_matrix=[]
time=[]
best_params=[]

#baseline parameters

p_grid = { 
    'var_smoothing': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
}

scoring = {"Accuracy": metrics.make_scorer(metrics.accuracy_score)}

#will contain the cv results
results = []

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(GaussianNB(),return_train_score=True,scoring="accuracy", refit=True, param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=3)
    grid.fit(X_train, y_train)
    
    
    #print("Best cross validation accuracy: {:.12f}".format(grid.best_score_))
    #print("Test set score: {:.12f}".format(grid.score(X_test,y_test)))
    #print("Best parameters: {}".format(grid.best_params_))
    #best_params.append(grid.best_params_)
    
    #Mas parametros a conseguir
    
    #y_pred=grid.predict(X_test)
    #accuracy.append(metrics.accuracy_score(y_test,y_pred))
    #precision.append(metrics.precision_score(y_test,y_pred))
    #recall.append(metrics.recall_score(y_test,y_pred))
    #f1_score.append(metrics.f1_score(y_test,y_pred))
    #confusion_matrix.append(metrics.confusion_matrix(y_test,y_pred))
    #parametros=grid.cv_results_['params']

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [16]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\GaussianGridSearchCV.csv", index = None, header=True) 

# RandomForest

In [17]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)


# define the model with default hyperparameters
model=RandomForestClassifier(random_state=42)

# define the grid of values to search
p_grid = { 
    'n_estimators': [100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [4,5],
    'criterion' :['gini', 'entropy']
}

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)
  

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [18]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\RForestGridSearchCV.csv", index = None, header=True) 

# Adaboost

In [19]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)


from sklearn.ensemble import AdaBoostClassifier
# define the model with default hyperparameters
model = AdaBoostClassifier()
# define the grid of values to search
p_grid = dict()
p_grid['n_estimators'] = [10,50,100]
p_grid['learning_rate'] = [0.1,0.001,0.0001]
scoring = {"AUC": "roc_auc", "Accuracy": metrics.make_scorer(metrics.accuracy_score), "Precision":metrics.make_scorer(metrics.precision_score)}

#will contain the cv results
results = []

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)
  

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [22]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\AdaboostGridSearchCV.csv", index = None, header=True) 

# LinearSVC

In [23]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

# define the model with default hyperparameters
model = LinearSVC()
# define the grid of values to search
p_grid = {'C':[0.1,10,100]}

#will contain the cv results
results = []

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=2, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)
  

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [24]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\LinearSVCGridSearchCV.csv", index = None, header=True) 

# Autoencoder

In [3]:
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers


X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

#Autoencoder
# Function to create model, required for KerasClassifier
def create_model():
    #Enc Dimension 
    encoding_dim=100
    #Input shape
    input_dim = Input(shape=(31,))
    #Encoding Layer
    encoded = Dense(encoding_dim, activation='relu')(input_dim)
    #Decoding Layer
    decoded = Dense(1, activation='sigmoid')(encoded)

    #Model AE
    autoencoder = Model(input_dim, decoded)
    #Model Encoder 
    encoder = Model(input_dim, encoded)
    #Encoding
    encoded_input = Input(shape=(encoding_dim,))
    #Decoding 
    decoder_layer = autoencoder.layers[-1]
    #Model Decoder 
    decoder = Model(encoded_input, decoder_layer(encoded_input))

    #optimizer=tf.keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=None, decay=0.0)
    autoencoder.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
    model = autoencoder

    # Compile model
    return model




model = KerasClassifier(build_fn = create_model)



# define the grid search parameters
#batch_size = [10, 20, 40]
#epochs = [10, 30]
#p_grid = dict(batch_size=batch_size, epochs=epochs)
#batch_size = [10,128]
#epochs = [10,50]


# define the grid search parameters
batch_size = [100, 200]
#optimizer= ["RMSprop", "adam"]
epochs = [10, 30]
#dropout = [0, 0.1, 0.2]
p_grid = dict(batch_size=batch_size, epochs=epochs)
p_grid = dict(batch_size=batch_size, epochs=epochs)


outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Epoch 1/30
2676/2676 [==============================] - 3s 924us/step - loss: 0.0113 - accuracy: 0.9976
Epoch 2/30
2676/2676 [==============================] - 2s 904us/step - loss: 5.6343e-04 - accuracy: 0.9999
Epoch 3/30
2676/2676 [==============================] - 2s 912us/step - loss: 1.7208e-04 - accuracy: 1.0000
Epoch 4/30
2676/2676 [==============================] - 2s 921us/step - loss: 7.6665e-05 - accuracy: 1.0000
Epoch 5/30
2676/2676 [==============================] - 2s 907us/step - loss: 4.7982e-05 - accuracy: 1.0000
Epoch 6/30
2676/2676 [==============================] - 2s 911us/step - loss: 1.7307e-04 - accuracy: 1.0000
Epoch 7/30
2676/2676 [==============================] - 2s 910us/step - loss: 7.7995e-05 - accuracy: 1.0000
Epoch 8/30
2676/2676 [==============================] - 2s 918us/step - loss: 3.5254e-06 - accuracy: 1.0000
Epoch 9/30
2676/2676 [==============================] - 2s 906us/step - loss: 1.

Epoch 5/10
2676/2676 [==============================] - 2s 910us/step - loss: 7.9866e-05 - accuracy: 1.0000
Epoch 6/10
2676/2676 [==============================] - 2s 904us/step - loss: 1.0037e-04 - accuracy: 1.0000
Epoch 7/10
2676/2676 [==============================] - 2s 903us/step - loss: 8.3061e-05 - accuracy: 1.0000
Epoch 8/10
2676/2676 [==============================] - 2s 910us/step - loss: 7.4729e-05 - accuracy: 1.0000
Epoch 9/10
2676/2676 [==============================] - 2s 903us/step - loss: 4.9428e-06 - accuracy: 1.0000
Epoch 10/10
2676/2676 [==============================] - 2s 902us/step - loss: 9.3234e-07 - accuracy: 1.0000
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Epoch 1/30
1338/1338 [==============================] - 2s 991us/step - loss: 0.0165 - accuracy: 0.9975
Epoch 2/30
1338/1338 [==============================] - 1s 991us/step - loss: 0.0011 - accuracy: 0.9998
Epoch 3/30
1338/1338 [==============================] - 1s 992us/step - loss: 4.663

In [4]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\AutoencoderGridSearchCV.csv", index = None, header=True) 

# TensorBoard tryout

In [18]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp


HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(X_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(X_test, y_test)
  return accuracy

session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
2090/2090 [==============================] - 2s 719us/step - loss: 0.0011 - accuracy: 0.9998
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
2090/2090 [==============================] - 2s 710us/step - loss: 0.0078 - accuracy: 0.9981
--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
2090/2090 [==============================] - 2s 726us/step - loss: 0.0013 - accuracy: 0.9991
--- Starting trial: run-3
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
2090/2090 [==============================] - 2s 728us/step - loss: 0.0077 - accuracy: 0.9980
--- Starting trial: run-4
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
2090/2090 [==============================] - 2s 723us/step - loss: 5.9942e-04 - accuracy: 0.9998
--- Starting trial: run-5
{'num_units'

In [19]:
%tensorboard --logdir logs/hparam_tuning



# RNN

In [3]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)


# Function to create model, required for KerasClassifier
# Initialising the RNN
def create_model():
    #Initializing the RNN
    model = Sequential()
    #Adding the first LSTM layer and dropout regularization
    model.add(LSTM(units=20, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0))
    #Adding the second LSTM layer and dropout regularization
    model.add(LSTM(units=20, return_sequences=True))
    model.add(Dropout(0))
    #Adding the output layer
    model.add(Dense(units=1))
    #Compiling the model without training
    model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn = create_model)

# define the grid search parameters
batch_size = [200]
#optimizer= ["RMSprop", "adam"]
epochs = [10, 20]
#dropout = [0, 0.1, 0.2]
p_grid = dict(batch_size=batch_size, epochs=epochs)


outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)



ERROR! Session/line number was not unique in database. History logging moved to new session 332
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Epoch 1/10
1338/1338 [==============================] - 44s 30ms/step - loss: 0.7420 - accuracy: 0.6656
Epoch 2/10
1338/1338 [==============================] - 41s 31ms/step - loss: 0.4967 - accuracy: 0.7942
Epoch 3/10
1338/1338 [==============================] - 41s 31ms/step - loss: 0.4809 - accuracy: 0.8130
Epoch 4/10
1338/1338 [==============================] - 41s 30ms/step - loss: 0.4764 - accuracy: 0.8185
Epoch 5/10
1338/1338 [==============================] - 41s 30ms/step - loss: 0.4748 - accuracy: 0.8185
Epoch 6/10
1338/1338 [==============================] - 41s 31ms/step - loss: 0.4745 - accuracy: 0.8185
Epoch 7/10
1338/1338 [==============================] - 41s 31ms/step - loss: 0.4743 - accuracy: 0.8185
Epoch 8/10
1338/1338 [==============================] - 41s 31ms/step - loss: 0.4742 - accuracy: 0.8185
Epoch 9/10
1

In [4]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\RNNGridSearchCV.csv", index = None, header=True) 

# Building the CNN


In [39]:
#Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

nX_train=X_train.to_numpy()
nX_train = nX_train[..., np.newaxis]
nX_test=X_test.to_numpy()
nX_test = nX_test[..., np.newaxis]


# Function to create model, required for KerasClassifier
# Initialising the CNN
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(nX_train.shape[1],nX_train.shape[2] )))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



model = KerasClassifier(build_fn = create_model)



# define the grid search parameters
batch_size = [32, 64]
#optimizer= ["RMSprop", "adam"]
epochs = [10, 20]
#dropout = [0, 0.1, 0.2]
p_grid = dict(batch_size=batch_size, epochs=epochs)



outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [ ]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\CNNGridSearchCV.csv", index = None, header=True) 

In [30]:
nX_train=X_train.to_numpy()
nX_train = nX_train[..., np.newaxis]

In [32]:
nX_train

array([[[-9.13798267e-01],
        [ 2.06517507e+00],
        [-6.23681245e-02],
        ...,
        [-1.60936040e+00],
        [-6.42964049e-01],
        [ 2.12768266e+00]],

       [[ 1.74578570e+00],
        [-4.03402134e-01],
        [-6.23681245e-02],
        ...,
        [ 6.21364859e-01],
        [-6.42964049e-01],
        [-4.69994900e-01]],

       [[ 1.46934696e+00],
        [-4.92854044e-01],
        [ 6.85065613e-04],
        ...,
        [ 6.21364859e-01],
        [-6.42964049e-01],
        [-4.69994900e-01]],

       ...,

       [[-9.13798267e-01],
        [ 2.10691519e+00],
        [-6.23681245e-02],
        ...,
        [-1.60936040e+00],
        [-6.42964049e-01],
        [ 2.12768266e+00]],

       [[ 1.04284389e+00],
        [ 1.56518800e-01],
        [-6.23681245e-02],
        ...,
        [ 6.21364859e-01],
        [-6.42964049e-01],
        [-4.69994900e-01]],

       [[ 8.18404276e-01],
        [-4.92854044e-01],
        [ 6.85065613e-04],
        ...,
        